# Test two Methods
* yfinance package
* and Tutorial from Medium

In [27]:
import yfinance as yf

dfgspc = yf.download('^GSPC', start="2020-08-01", end="2020-08-26")

[*********************100%***********************]  1 of 1 completed


In [28]:
dfgspc.tail(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-08-21,3386.010010,3399.959961,3379.310059,3397.159912,3397.159912,3705420000
2020-08-24,3418.090088,3432.090088,3413.129883,3431.280029,3431.280029,3728690000
2020-08-25,3435.949951,3439.159912,3425.840088,3436.110107,3436.110107,1215658829


# Tutorial from Medium article
Author Debra Ray

* **Part 1** Request Headers

* **Part 2** Python Libraries, Modules, Packages and Scraping Libraries 

* **Part 3** The Yahoo Finance Price History Scraper

Note: The sections below names are named to provide a better function description of the content

### Part 1 Request Headers

* Your header should be a dictionary object with the keys set to pre-defined information that servers expect to receive in HTTP requests. You will need to define the key values in your header. An example of an appropriate and complete header is:  
'''
hdrs =  {"authority": "finance.yahoo.com",
            "method": "GET",
            "path": subdoma.format(symbol, start, end),
            "scheme": "https",
            "accept": "text/html",
            "accept-encoding": "gzip, deflate, br",
            "accept-language": "en-US,en;q=0.9",
            "cache-control": "no-cache",
            "cookie": "Cookie:identifier",
            "dnt": 1,
            "pragma": "no-cache",
            "sec-fetch-mode": "navigate",
            "sec-fetch-site": "same-origin",
            "sec-fetch-user": "?1",
            "upgrade-insecure-requests": 1,
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64)"}
'''

* **Request Headers Should Include a Defined Path**
  * Take note of the path key in the header example above. The key-value pair for the header is {“path”: subdoma.format(symbol, start, end)}. Every Yahoo subdomain takes a set of arguments as parameters to guide you to the right page. In the case of stock price data the arguments are as follows:
  * Quick Note on time frame inputs: Yahoo only returns 100 trading day intervals per scraping request. Even if you choose a start and end date longer than 100 trading days, you will only receive 100 trading days worth of data in return via the scraper. We will cover how to scrape more than 100 days worth of data efficiently in a future article.
  
  * Take note of the path key in the header example above. The key-value pair for the header is {“path”: subdoma.format(symbol, start, end)}. Every Yahoo subdomain takes a set of arguments as parameters to guide you to the right page. In the case of stock price data the arguments are as follows:
     * Stock symbol: The symbol of the stock you want to retrieve
     * Start date: The earliest trading day that you would like to retrieve.
     * End date: The latest trading day that you would like to retrieve.
     * Subdomain filter: In this exercise, we will use the “History” filter.
     * Interval: Timeframe to calculate high, low, open, and closing stock prices
     * Frequency: Same as the interval argument.
  * When you define your headers for Yahoo, you will want to create a header object customized to the symbol you are scraping, the time frame in which you would like the data, and the subdomain filter.

### Part 2 Python Libraries, Modules, Packages and Scraping Libraries  


Defining Libraries, Packages, Modules, Methods, and Functions
* **Libraries:** A collection of functions and methods to perform a group of tasks like data analysis or data visualization.
* **Packages:** Groups of modules combined with an __init__.py file for distribution via a software repository.
* **Modules:** Python files, with the extension .py, that comprise Python packages and libraries and contain methods and functions.
* **Methods:** Functions within class objects that can only be implemented on the class objects with which the functions are associated.
* *Functions:** A set of commands to perform a defined task outside of an object class, which begins with def function_name(*args, **kwargs).
    
Libraries for Scraping

* **requests:** An HTTP library for Python, which allows you to make requests to specified URLs and receive a response from the URL. Learn more about HTTP requests here.
* **pandas:** A data analysis library for Python that can read HTML tables and turn them into a list of data frames. Read more about the HTML parsing functionality here.
* **lxml:** Parses website data into dictionary objects from element trees or directories of information contained on a webpage. Find a more in-depth explanation here.




### References

**Prevent from getting black listed while scraping**  
https://www.scrapehero.com/how-to-prevent-getting-blacklisted-while-scraping/

**Xpath**   
http://courses.ischool.berkeley.edu/i290-14/s05/lecture-4/allslides.html

### Part 3 

In [16]:
# note, the complet program is contained in the file fmscraper.py
# Part 3
from datetime import datetime, timedelta 
import time
import requests, pandas, lxml
from lxml import html

def format_date2(date_datetime):
     date_timetuple = date_datetime.timetuple()
     date_mktime = time.mktime(date_timetuple)
     date_int = int(date_mktime)
     date_str = str(date_int)
     return date_str
    
def subdomain2(symbol, start, end, filter='history'):
    subdoma="/quote/{0}/history?period1={1}&period2={2}&interval=1d&filter={3}&frequency=1d"
    subdomain = subdoma.format(symbol, start, end, filter)
    return subdomain
    
def header_function2(subdomain):
     hdrs =  {"authority": "finance.yahoo.com",
              "method": "GET",
              "path": subdomain,
              "scheme": "https",
              "accept": "text/html",
              "accept-encoding": "gzip, deflate, br",
              "accept-language": "en-US,en;q=0.9",
              "cache-control": "no-cache",
              "cookie": "Cookie:identifier",
              "dnt": "1",
              "pragma": "no-cache",
              "sec-fetch-mode": "navigate",
              "sec-fetch-site": "same-origin",
              "sec-fetch-user": "?1",
              "upgrade-insecure-requests": "1",
              "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64)"}
     
     return hdrs
    
def scrape_page2(url, header):
     page = requests.get(url, headers=header)
     element_html = html.fromstring(page.content)
     table = element_html.xpath('//table')
     # put into byte string with lxml.etree.tostring
     #  xpath will return list of tables ... we only need the first one 
     table_tree = lxml.etree.tostring(table[0], method='xml')
     # for some reason we are getting a list of which the 1st element is the df
     # we are getting a weird comment row at the end
    
    # Yahoo limits the number of days downloaded to 100
     panda = pandas.read_html(table_tree)[0]

    # to complet the example limit to 0:99 ... need to generalize this for real code
     panda = panda.loc[0:99]
     return panda

if __name__ == '__main__':
     symbol = 'BB'
     
     dt_start = datetime.today() - timedelta(days=365)
     dt_end = datetime.today()
    
     start = format_date2(dt_start)
     end = format_date2(dt_end)
     
     sub = subdomain2(symbol, start, end)
     header = header_function2(sub)
     
     base_url = 'https://finance.yahoo.com'
     url = base_url + sub
     price_history = scrape_page2(url, header)

In [18]:

symbol='BB'
# max rows downloaded will be 100 though we are indicated more
dt_start = datetime.today() - timedelta(days=365)
dt_end = datetime.today()

start = format_date2(dt_start)
end = format_date2(dt_end)
     
sub = subdomain2(symbol, start, end)
header = header_function2(sub)
     
base_url = 'https://finance.yahoo.com'
url = base_url + sub
dfpricehistory = scrape_page2(url, header)
#dfhistory=price_history[0]
display(dfpricehistory)

,Date,Open,High,Low,Close*,Adj Close**,Volume
0,"Aug 24, 2020",4.8000,4.8400,4.7420,4.7900,4.7900,2515622
1,"Aug 21, 2020",4.8000,4.8400,4.7300,4.7500,4.7500,2037600
2,"Aug 20, 2020",4.7400,4.8500,4.7200,4.8300,4.8300,2103300
3,"Aug 19, 2020",4.8400,4.8700,4.7500,4.7800,4.7800,2641700
4,"Aug 18, 2020",4.8100,4.8600,4.7400,4.7900,4.7900,2432900
...,...,...,...,...,...,...,...
95,"Apr 08, 2020",3.7300,3.8800,3.6400,3.8400,3.8400,6287500
96,"Apr 07, 2020",3.6700,3.7900,3.5900,3.6100,3.6100,6674600
97,"Apr 06, 2020",3.3000,3.5300,3.2500,3.4800,3.4800,10518100
98,"Apr 03, 2020",3.2200,3.3500,3.1800,3.2200,3.2200,5268100


In [21]:

%run fmfunctions
symbol='BB'
# max rows downloaded will be 100 though we are indicated more
dt_start = dt.datetime.today() - dt.timedelta(days=365)
dt_end = dt.datetime.today()

start = format_date(dt_start)
end = format_date(dt_end)
     
sub = subdomain(symbol, start, end)
header = header_function2(sub)
     
base_url = 'https://finance.yahoo.com'
url = base_url + sub
dfpricehistory = scrape_page(url, header)
#dfhistory=price_history[0]
display(dfpricehistory.loc[0:99])

,Date,Open,High,Low,Close,Adj CLose,Volume
0,"Aug 24, 2020",4.8000,4.8400,4.7420,4.7900,4.7900,2515622
1,"Aug 21, 2020",4.8000,4.8400,4.7300,4.7500,4.7500,2037600
2,"Aug 20, 2020",4.7400,4.8500,4.7200,4.8300,4.8300,2103300
3,"Aug 19, 2020",4.8400,4.8700,4.7500,4.7800,4.7800,2641700
4,"Aug 18, 2020",4.8100,4.8600,4.7400,4.7900,4.7900,2432900
...,...,...,...,...,...,...,...
95,"Apr 08, 2020",3.7300,3.8800,3.6400,3.8400,3.8400,6287500
96,"Apr 07, 2020",3.6700,3.7900,3.5900,3.6100,3.6100,6674600
97,"Apr 06, 2020",3.3000,3.5300,3.2500,3.4800,3.4800,10518100
98,"Apr 03, 2020",3.2200,3.3500,3.1800,3.2200,3.2200,5268100
